In [13]:
import tensorflow as tf
from pathlib import Path
from IPython.display import Image
import numpy as np
from tensorflow.keras.layers import Conv2D, LeakyReLU, concatenate, UpSampling2D

import sys
working_directory = str(Path.home() / 'projects/cs230_project_keras/skin_lesion')
sys.path.append(working_directory)

import utils
from model.full_unet import FullUnet

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


So now let's build UNet with VGG-16 encoder. This means that the left part of UNet will be replaced by VGG-16.

## `VGG-16`

In [7]:
data_dir  = Path('../experiments/transf_learn_vgg_toy')

In [8]:
params = utils.Params(data_dir / 'params.json')

In [9]:
params.input_shape

[256, 256, 3]

In [10]:
base_model = tf.keras.applications.VGG16(weights='imagenet',
                                         include_top=False,
                                         input_shape=params.input_shape)

In [11]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

## combined model

In [16]:
for layer in base_model.layers:
    layer.trainable = False

In [17]:
x = base_model.output

bottom_conv = Conv2D(1024, (3, 3), padding="same")(x)
bottom_acti = LeakyReLU(alpha=0.001)(bottom_conv)
bottom_conv = Conv2D(1024, (3, 3), padding="same")(bottom_acti)
bottom_acti = LeakyReLU(alpha=0.001)(bottom_conv)

# 14 x 14 block5
left_block5 = base_model.get_layer('block5_conv3').output
right_up5 = concatenate([UpSampling2D(size=(2, 2))(bottom_acti), left_block5], axis=3)
right_conv5 = Conv2D(512, (3, 3), padding="same")(right_up5)
right_acti5 = LeakyReLU(alpha=0.001)(right_conv5)
right_conv5 = Conv2D(512, (3, 3), padding="same")(right_acti5)
right_acti5 = LeakyReLU(alpha=0.001)(right_conv5)

# 28 x 28 block4
left_block4 = base_model.get_layer('block4_conv3').output
right_up4 = concatenate([UpSampling2D(size=(2, 2))(right_acti5), left_block4], axis=3)
right_conv4 = Conv2D(512, (3, 3), padding="same")(right_up4)
right_acti4 = LeakyReLU(alpha=0.001)(right_conv4)
right_conv4 = Conv2D(512, (3, 3), padding="same")(right_acti4)
right_acti4 = LeakyReLU(alpha=0.001)(right_conv4)

# 56 x 56 block3
left_block3 = base_model.get_layer('block3_conv3').output
right_up3 = concatenate([UpSampling2D(size=(2, 2))(right_acti4), left_block3], axis=3)
right_conv3 = Conv2D(256, (3, 3), padding="same")(right_up3)
right_acti3 = LeakyReLU(alpha=0.001)(right_conv3)
right_conv3 = Conv2D(256, (3, 3), padding="same")(right_acti3)
right_acti3 = LeakyReLU(alpha=0.001)(right_conv3)

# 112 x 112 block2
left_block2 = base_model.get_layer('block2_conv2').output
right_up2 = concatenate([UpSampling2D(size=(2, 2))(right_acti3), left_block2], axis=3)
right_conv2 = Conv2D(128, (3, 3), padding="same")(right_up2)
right_acti2 = LeakyReLU(alpha=0.001)(right_conv2)
right_conv2 = Conv2D(128, (3, 3), padding="same")(right_acti2)
right_acti2 = LeakyReLU(alpha=0.001)(right_conv2)

# 224 x 224 block1
left_block1 = base_model.get_layer('block1_conv2').output
right_up1 = concatenate([UpSampling2D(size=(2, 2))(right_acti2), left_block1], axis=3)
right_conv1 = Conv2D(64, (3, 3), padding="same")(right_up1)
right_acti1 = LeakyReLU(alpha=0.001)(right_conv1)
right_conv1 = Conv2D(64, (3, 3), padding="same")(right_acti1)
right_acti1 = LeakyReLU(alpha=0.001)(right_conv1)


#Sigmoid
output = Conv2D(1, (1, 1), activation='sigmoid')(right_acti1)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 256, 256, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 128, 128, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________